In [1]:
import os
import pandas as pd
pd.options.mode.copy_on_write = True

In [2]:
data_dir = 'data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
if os.path.exists(f'{data_dir}/nasdaqlisted.csv'):
    print('Nasdaq info already downloaded.')
else:
    print('Downloading nasdaq info...')
    try:
        os.system(f'wget http://www.nasdaqtrader.com/dynamic/symdir/nasdaqlisted.txt -O {data_dir}/nasdaqlisted.csv')
    except Exception as e:
        print('Failed to download nasdaq info due to error:')
        print(e)

Nasdaq info already downloaded.


In [3]:
nasdaq = pd.read_csv(f'{data_dir}/nasdaqlisted.csv', sep='|')
nasdaq

,Symbol,Security Name,Market Category,Test Issue,Financial Status,Round Lot Size,ETF,NextShares
0,AACG,ATA Creativity Global - American Depositary Sh...,G,N,N,100.0,N,N
1,AACI,Armada Acquisition Corp. I - Common Stock,G,N,N,100.0,N,N
2,AACIU,Armada Acquisition Corp. I - Unit,G,N,N,100.0,N,N
3,AACIW,Armada Acquisition Corp. I - Warrant,G,N,N,100.0,N,N
4,AADI,"Aadi Bioscience, Inc. - Common Stock",S,N,N,100.0,N,N
...,...,...,...,...,...,...,...,...
4856,ZXZZT,NASDAQ TEST STOCK,G,Y,N,100.0,NaN,N
4857,ZYME,Zymeworks Inc. - Common Stock,Q,N,N,100.0,N,N
4858,ZYXI,"Zynex, Inc. - Common Stock",Q,N,N,100.0,N,N
4859,ZZZ,Cyber Hornet S&P 500 and Bitcoin 75/25 Strateg...,G,N,N,100.0,Y,N


# Cleaning time...

## First off, the last row is garbage and should be removed

In [4]:
nasdaq = nasdaq[:-1]
nasdaq

,Symbol,Security Name,Market Category,Test Issue,Financial Status,Round Lot Size,ETF,NextShares
0,AACG,ATA Creativity Global - American Depositary Sh...,G,N,N,100.0,N,N
1,AACI,Armada Acquisition Corp. I - Common Stock,G,N,N,100.0,N,N
2,AACIU,Armada Acquisition Corp. I - Unit,G,N,N,100.0,N,N
3,AACIW,Armada Acquisition Corp. I - Warrant,G,N,N,100.0,N,N
4,AADI,"Aadi Bioscience, Inc. - Common Stock",S,N,N,100.0,N,N
...,...,...,...,...,...,...,...,...
4855,ZXYZ.A,Nasdaq Symbology Test Common Stock,Q,Y,N,100.0,NaN,N
4856,ZXZZT,NASDAQ TEST STOCK,G,Y,N,100.0,NaN,N
4857,ZYME,Zymeworks Inc. - Common Stock,Q,N,N,100.0,N,N
4858,ZYXI,"Zynex, Inc. - Common Stock",Q,N,N,100.0,N,N


## Second, some of the columns may not be useful, so let's investigate them starting with the `Market Category` column

In [5]:
nasdaq['Market Category'].unique()

array(['G', 'S', 'Q'], dtype=object)

The following is the definition of the market category field from https://www.nasdaqtrader.com/Trader.aspx?id=SymbolDirDefs :

_The category assigned to the issue by NASDAQ based on Listing Requirements._
    
_Values:_
- _Q = NASDAQ Global Select MarketSM_
- _G = NASDAQ Global MarketSM_
- _S = NASDAQ Capital Market_

---

Given the above description, it should be easy enough to just map the values to the corresponding categories.

In [6]:
market_category_map = {
    'Q': 'NASDAQ Global Select Market',
    'G': 'NASDAQ Global Market',
    'S': 'NASDAQ Capital Market',
}
nasdaq['Market Category'] = nasdaq['Market Category'].map(lambda code: market_category_map[code])
nasdaq

,Symbol,Security Name,Market Category,Test Issue,Financial Status,Round Lot Size,ETF,NextShares
0,AACG,ATA Creativity Global - American Depositary Sh...,NASDAQ Global Market,N,N,100.0,N,N
1,AACI,Armada Acquisition Corp. I - Common Stock,NASDAQ Global Market,N,N,100.0,N,N
2,AACIU,Armada Acquisition Corp. I - Unit,NASDAQ Global Market,N,N,100.0,N,N
3,AACIW,Armada Acquisition Corp. I - Warrant,NASDAQ Global Market,N,N,100.0,N,N
4,AADI,"Aadi Bioscience, Inc. - Common Stock",NASDAQ Capital Market,N,N,100.0,N,N
...,...,...,...,...,...,...,...,...
4855,ZXYZ.A,Nasdaq Symbology Test Common Stock,NASDAQ Global Select Market,Y,N,100.0,NaN,N
4856,ZXZZT,NASDAQ TEST STOCK,NASDAQ Global Market,Y,N,100.0,NaN,N
4857,ZYME,Zymeworks Inc. - Common Stock,NASDAQ Global Select Market,N,N,100.0,N,N
4858,ZYXI,"Zynex, Inc. - Common Stock",NASDAQ Global Select Market,N,N,100.0,N,N


Nice, thats looking better already. 
## Let's move on to the next column, `Test Issue`.

In [7]:
nasdaq['Test Issue'].unique()

array(['N', 'Y'], dtype=object)

From the same source as above, the definition of the `Test Issue` field is:

_Indicates whether or not the security is a test security. Values: Y = yes, it is a test issue. N = no, it is not a test issue._

---
In this case, I think it would be best to just remove records where the `Test Issue` field is `Y`, but first let's just have a quick look at the data to see what those records look like. 

In [8]:
nasdaq[nasdaq['Test Issue'] == 'Y']

,Symbol,Security Name,Market Category,Test Issue,Financial Status,Round Lot Size,ETF,NextShares
4813,ZAZZT,Tick Pilot Test Stock Class A Common Stock,NASDAQ Global Market,Y,N,100.0,NaN,N
4816,ZBZZT,Test Pilot Test Stock Class B Common Stock,NASDAQ Global Market,Y,N,100.0,NaN,N
4820,ZCZZT,Tick Pilot Test Stock Class C,NASDAQ Global Market,Y,N,100.0,NaN,N
4834,ZJZZT,NASDAQ TEST STOCK,NASDAQ Global Select Market,Y,N,100.0,NaN,N
4853,ZVZZT,NASDAQ TEST STOCK,NASDAQ Global Market,Y,N,100.0,NaN,N
4854,ZWZZT,NASDAQ TEST STOCK,NASDAQ Capital Market,Y,N,100.0,NaN,N
4855,ZXYZ.A,Nasdaq Symbology Test Common Stock,NASDAQ Global Select Market,Y,N,100.0,NaN,N
4856,ZXZZT,NASDAQ TEST STOCK,NASDAQ Global Market,Y,N,100.0,NaN,N


As expected, the records with `Test Issue` set to `Y` are not useful, so let's remove them as well as the test issue column. 

In [9]:
nasdaq = nasdaq[nasdaq['Test Issue'] == 'N']
nasdaq = nasdaq.drop(columns=['Test Issue'])
nasdaq

,Symbol,Security Name,Market Category,Financial Status,Round Lot Size,ETF,NextShares
0,AACG,ATA Creativity Global - American Depositary Sh...,NASDAQ Global Market,N,100.0,N,N
1,AACI,Armada Acquisition Corp. I - Common Stock,NASDAQ Global Market,N,100.0,N,N
2,AACIU,Armada Acquisition Corp. I - Unit,NASDAQ Global Market,N,100.0,N,N
3,AACIW,Armada Acquisition Corp. I - Warrant,NASDAQ Global Market,N,100.0,N,N
4,AADI,"Aadi Bioscience, Inc. - Common Stock",NASDAQ Capital Market,N,100.0,N,N
...,...,...,...,...,...,...,...
4851,ZVRA,"Zevra Therapeutics, Inc. - Common Stock",NASDAQ Global Select Market,N,100.0,N,N
4852,ZVSA,"ZyVersa Therapeutics, Inc. - Common Stock",NASDAQ Capital Market,N,100.0,N,N
4857,ZYME,Zymeworks Inc. - Common Stock,NASDAQ Global Select Market,N,100.0,N,N
4858,ZYXI,"Zynex, Inc. - Common Stock",NASDAQ Global Select Market,N,100.0,N,N


## Next up, the `Financial Status` column.

From the same source as above, the definition of the `Financial Status` field is:

_Indicates when an issuer has failed to submit its regulatory filings on a timely basis, has failed to meet NASDAQ's continuing listing standards, and/or has filed for bankruptcy. Values include:_

- _D = Deficient: Issuer Failed to Meet NASDAQ Continued Listing Requirements_
- _E = Delinquent: Issuer Missed Regulatory Filing Deadline_
- _Q = Bankrupt: Issuer Has Filed for Bankruptcy_
- _N = Normal (Default): Issuer Is NOT Deficient, Delinquent, or Bankrupt._
- _G = Deficient and Bankrupt_
- _H = Deficient and Delinquent_
- _J = Delinquent and Bankrupt_
- _K = Deficient, Delinquent, and Bankrupt_

---
Given the above description, we can do something similar to what we did with the `Market Category` column, and map the values to more descriptive categories.

In [10]:
financial_status_map = {
    'D': 'Deficient',
    'E': 'Delinquent',
    'Q': 'Bankrupt',
    'N': 'Normal',
    'G': 'Deficient and Bankrupt',
    'H': 'Deficient and Delinquent',
    'J': 'Delinquent and Bankrupt',
    'K': 'Deficient, Delinquent, and Bankrupt',
}
nasdaq['Financial Status'] = nasdaq['Financial Status'].map(lambda code: financial_status_map[code])
nasdaq

,Symbol,Security Name,Market Category,Financial Status,Round Lot Size,ETF,NextShares
0,AACG,ATA Creativity Global - American Depositary Sh...,NASDAQ Global Market,Normal,100.0,N,N
1,AACI,Armada Acquisition Corp. I - Common Stock,NASDAQ Global Market,Normal,100.0,N,N
2,AACIU,Armada Acquisition Corp. I - Unit,NASDAQ Global Market,Normal,100.0,N,N
3,AACIW,Armada Acquisition Corp. I - Warrant,NASDAQ Global Market,Normal,100.0,N,N
4,AADI,"Aadi Bioscience, Inc. - Common Stock",NASDAQ Capital Market,Normal,100.0,N,N
...,...,...,...,...,...,...,...
4851,ZVRA,"Zevra Therapeutics, Inc. - Common Stock",NASDAQ Global Select Market,Normal,100.0,N,N
4852,ZVSA,"ZyVersa Therapeutics, Inc. - Common Stock",NASDAQ Capital Market,Normal,100.0,N,N
4857,ZYME,Zymeworks Inc. - Common Stock,NASDAQ Global Select Market,Normal,100.0,N,N
4858,ZYXI,"Zynex, Inc. - Common Stock",NASDAQ Global Select Market,Normal,100.0,N,N


I think I would also like to keep the definition of the `Financial Status` values on hand for future reference, so I will keep them in a csv file for now.

In [11]:
financial_status_definition_map = {
    'Deficient': 'Issuer Failed to Meet NASDAQ Continued Listing Requirements',
    'Delinquent': 'Issuer Missed Regulatory Filing Deadline',
    'Bankrupt': 'Issuer Has Filed for Bankruptcy',
    'Normal': 'Issuer Is NOT Deficient, Delinquent, or Bankrupt'
}
financial_status_definition_map_df = pd.DataFrame.from_dict(financial_status_definition_map, orient='index', columns=['Definition'])
financial_status_definition_map_df.to_csv(f'{data_dir}/financial_status_definition_map.csv')
financial_status_definition_map_df

,Definition
Deficient,Issuer Failed to Meet NASDAQ Continued Listing...
Delinquent,Issuer Missed Regulatory Filing Deadline
Bankrupt,Issuer Has Filed for Bankruptcy
Normal,"Issuer Is NOT Deficient, Delinquent, or Bankrupt"


## Now, for the `Round Lot Size` column.

From the same source as above, the definition of the `Round Lot Size` field is:

_Indicates the number of shares that make up a round lot for the given security._

---
I don't really understand what that means, so let's just have a quick look at the data to see what it looks like.

In [12]:
nasdaq['Round Lot Size'].unique()

array([100.])

To me, it looks like the `Round Lot Size` column is not useful, so I will remove it.

In [13]:
nasdaq = nasdaq.drop(columns=['Round Lot Size'])

## Lastly, I think the `ETF` and `NextShares` columns can be dealt with together.

First, since they are not listed in the source above, I'm currently just assuming that they're boolean values, but let's just have a quick look at the data to see what they look like.

In [14]:
nasdaq['ETF'].unique()

array(['N', 'Y'], dtype=object)

In [15]:
nasdaq['NextShares'].unique()

array(['N'], dtype=object)

Okay, so it looks like the `ETF` and `NextShares` columns are boolean values. That being said, since `NextShares` are all `N`, I think it would be best to remove that column, and convert the `ETF` column to 0s and 1s.

In [16]:
nasdaq = nasdaq.drop(columns=['NextShares'])
nasdaq.loc[nasdaq['ETF'] == 'Y', 'ETF'] = 1
nasdaq.loc[nasdaq['ETF'] == 'N', 'ETF'] = 0
nasdaq

,Symbol,Security Name,Market Category,Financial Status,ETF
0,AACG,ATA Creativity Global - American Depositary Sh...,NASDAQ Global Market,Normal,0
1,AACI,Armada Acquisition Corp. I - Common Stock,NASDAQ Global Market,Normal,0
2,AACIU,Armada Acquisition Corp. I - Unit,NASDAQ Global Market,Normal,0
3,AACIW,Armada Acquisition Corp. I - Warrant,NASDAQ Global Market,Normal,0
4,AADI,"Aadi Bioscience, Inc. - Common Stock",NASDAQ Capital Market,Normal,0
...,...,...,...,...,...
4851,ZVRA,"Zevra Therapeutics, Inc. - Common Stock",NASDAQ Global Select Market,Normal,0
4852,ZVSA,"ZyVersa Therapeutics, Inc. - Common Stock",NASDAQ Capital Market,Normal,0
4857,ZYME,Zymeworks Inc. - Common Stock,NASDAQ Global Select Market,Normal,0
4858,ZYXI,"Zynex, Inc. - Common Stock",NASDAQ Global Select Market,Normal,0


## Now all that's left is to save the cleaned data to a new csv file for later use.

In [17]:
nasdaq.to_csv(f'{data_dir}/nasdaq_cleaned.csv', index=False)